In [1]:
import pandas as pd
from sklearn.svm import SVR
from tqdm import tqdm_notebook as tqdm
import numpy as np
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
import warnings;warnings.simplefilter('ignore')
from sklearn.pipeline import Pipeline
import joblib

In [2]:
def mape(a, b):
    a = np.array(a)
    b = np.array(b)
    mask = a != 0
    return (np.fabs(a-b)/a)[mask].mean()

def metric(y_t,y_p,name):
  res = {'R2' : max(r2_score(y_t,y_p), 0),
         'RMSE': sqrt(mean_squared_error(y_t,y_p)),
         'MAPE': mape(y_t,y_p)}
  return pd.DataFrame(res,index=[name])

# load data

In [3]:
def load_data():
  train = pd.read_csv('../data/phase_1/train_4565.csv',index_col=0)
  test = pd.read_csv('../data/phase_1/test_170.csv',index_col=0)
  
  # add P = 100 - (N+A)
  train['P'] = 100 - train['N+A']
  test['P'] = 100 - test['N+A']
  
  # define columns
  x_cols = ['T10','T50','T90','N+A','P']
  y_cols = ['C5NP','C5IP','C5N','C6NP','C6IP','C6N','C6A','C7NP','C7IP','C7N','C7A',
            'C8NP','C8IP','C8N','C8A','C9NP','C9IP','C9N','C9A','C10NP','C10IP','C10N','C10A']
  N_col = ['C5N','C6N','C6A','C7N','C7A','C8N','C8A','C9N','C9A','C10N','C10A']
  P_col = ['C5NP','C5IP','C6NP','C6IP','C7NP','C7IP','C8NP','C8IP','C9NP','C9IP','C10NP','C10IP']
  
  # split_data train and test
  X_train = train[x_cols]
  X_test = test[x_cols]
  y_train = train[y_cols]
  y_test = test[y_cols]
  
  return X_train,y_train,X_test,y_test

In [4]:
X_train,y_train,X_test,y_test = load_data()
x_cols = X_train.columns.tolist()
y_cols = y_train.columns.tolist()

# define model

In [5]:
class custom_model(object):
  def __init__(self,x_cols,y_cols):
    self.x_cols = x_cols
    self.y_cols = y_cols
    self.N_col = ['C5N','C6N','C6A','C7N','C7A','C8N','C8A','C9N','C9A','C10N','C10A']
    self.P_col = ['C5NP','C5IP','C6NP','C6IP','C7NP','C7IP','C8NP','C8IP','C9NP','C9IP','C10NP','C10IP']
    self.model_23 = {}
    for y_name in y_cols:
      self.model_23[y_name] = Pipeline([('scaler',StandardScaler()),('reg',SVR(C=0.3))])
  
  def fit(self,X,y):
    for y_name in tqdm(self.y_cols):
      self.model_23[y_name].fit(X,y[y_name])
      y_pred = self.model_23[y_name].predict(X) 
      # Sequence prediction add y_pred to X 
      X.loc[:,y_name] = y_pred
    # recover X
    X = X[self.x_cols]
  
  def predict(self,data):
    X = data.copy()    
    results = pd.DataFrame(index=[*range(len(X))],columns=self.y_cols)
    for y_name in self.y_cols:
      y_pred = self.model_23[y_name].predict(X)
      results.loc[:,y_name] = y_pred
      # Sequence prediction add y_pred to X 
      X.loc[:,y_name] = y_pred
    # recover X
    X = X[self.x_cols]
    
    # normalize depand on N+A and P
    results[self.N_col] = self._normalize(results[self.N_col])*X['N+A'].values.reshape(-1,1)
    results[self.P_col] = self._normalize(results[self.P_col])*X['P'].values.reshape(-1,1)

    return results.values
  
  @staticmethod
  def _normalize(x):
    return x/x.sum(axis=1).values.reshape(-1,1)

# fit model

In [6]:
model = custom_model(x_cols,y_cols)
model.fit(X_train,y_train)

# make predictions

In [7]:
y_pred = model.predict(X_test)
y_pred = pd.DataFrame(y_pred,index=[*range(len(X_test))],columns=y_cols)

In [8]:
y_pred.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,...,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.203686,0.819795,0.162816,3.601440,2.287253,3.558247,0.671907,8.028073,6.381046,9.598324,...,7.060207,5.897693,5.231254,9.390761,4.992742,3.516019,1.148820,5.289111,0.680566,0.697045
1,0.139352,0.104192,0.102326,3.308866,1.101464,2.172174,0.400285,10.542947,7.665591,6.351698,...,4.990709,5.599301,6.804631,10.796959,4.466295,3.895169,1.195626,6.076774,0.602062,0.704837
2,0.127106,0.090924,0.104198,3.324566,1.111368,2.099043,0.373143,10.585076,7.645596,6.088840,...,4.790175,5.451137,6.921815,10.855879,4.338635,3.933121,1.333154,6.310952,0.602479,0.845768
3,0.117018,0.090114,0.103041,3.450313,1.181297,2.107131,0.412052,10.573413,7.545769,6.058131,...,4.828324,5.559203,6.941416,10.916143,4.370726,3.929430,1.209418,6.229487,0.548480,0.635649
4,0.153042,0.112315,0.102393,3.262805,1.117298,2.159606,0.408213,10.380394,7.477592,6.314363,...,5.017444,5.702095,6.833520,10.830178,4.461771,4.028774,1.236873,6.242840,0.594647,0.735394


In [9]:
y_test.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,...,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.142,0.616,0.217,3.745,2.577,4.028,0.557,7.669,5.9900,10.206,...,7.039,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597
1,0.122,0.078,0.029,3.955,1.563,2.407,0.400,10.016,7.2633,6.573,...,5.135,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484
2,0.098,0.064,0.020,3.920,1.447,2.296,0.396,10.236,7.3480,6.259,...,4.942,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456
3,0.122,0.085,0.021,3.900,1.437,2.274,0.397,10.259,7.3560,6.234,...,4.928,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449
4,0.072,0.049,0.015,3.873,1.369,2.206,0.489,10.213,7.3190,6.009,...,4.769,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463


# metrics

In [10]:
N_col = ['C5N','C6N','C6A','C7N','C7A','C8N','C8A','C9N','C9A','C10N','C10A']
temp = pd.DataFrame()
temp['real_N+A'] = X_test['N+A'].values
temp['pred_N+A'] = y_pred[N_col].sum(axis=1).values
temp

,real_N+A,pred_N+A
0,40.133,40.133
1,32.175,32.175
2,31.428,31.428
3,31.390,31.390
4,32.418,32.418
...,...,...
165,53.841,53.841
166,53.442,53.442
167,52.549,52.549
168,49.827,49.827


In [11]:
res = pd.DataFrame()
for y_name in y_cols:
  res = res.append(metric(y_test[y_name],y_pred[y_name],y_name))
res.loc['AVG'] = res.mean()

In [12]:
res

,R2,RMSE,MAPE
C5NP,0.873977,0.303375,0.286513
C5IP,0.747573,0.257102,0.477437
C5N,0.791451,0.062517,2.210790
C6NP,0.268743,0.345968,0.063613
C6IP,0.881104,0.360419,0.093511
C6N,0.887215,0.556155,0.052337
C6A,0.491827,0.121792,0.173971
C7NP,0.971814,0.369232,0.043670
C7IP,0.852944,0.423746,0.049250
C7N,0.982487,0.551500,0.033404


# check if pass?

In [13]:
c1 = (len(res.loc[res.RMSE > 0.6]) == 0)
c2 = (res.loc['AVG','RMSE'] < 0.4)
print(c1,c2)

True True


# save model

In [14]:
joblib.dump(model,'../model/SVR(5_to_23).pkl')
print('save done')

save done
